In [1]:
import torch
import clip
from PIL import Image
import torch

/ckb-nfs/home/zcafego/git/openmpf-projects/openmpf-components/python/ClipDetection/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda:2'

vitl14_path = './vit_l14_ep50_16shots/nctx16_cscFalse_ctpend/seed3/prompt_learner/model.pth.tar-50'
vitb32_path = './vit_b32_ep50_1shots/nctx16_cscFalse_ctpend/seed3/prompt_learner/model.pth.tar-50'

vitl14_check = torch.load(vitl14_path, map_location='cpu')
vitb32_check = torch.load(vitb32_path, map_location='cpu')

vitl14_state_dict = vitl14_check['state_dict']
vitl14_tnsr = vitl14_state_dict['ctx']

vitb32_state_dict = vitb32_check['state_dict']
vitb32_tnsr = vitb32_state_dict['ctx']

img = Image.open('/ckb-nfs/home/zcafego/test_images/sturgeon.JPEG')

In [3]:
l14_model, l14_preprocessor = clip.load('ViT-L/14', device=device)

In [4]:
l14_model.load_state_dict(vitl14_state_dict, strict=False)

_IncompatibleKeys(missing_keys=['positional_embedding', 'text_projection', 'logit_scale', 'visual.class_embedding', 'visual.positional_embedding', 'visual.proj', 'visual.conv1.weight', 'visual.ln_pre.weight', 'visual.ln_pre.bias', 'visual.transformer.resblocks.0.attn.in_proj_weight', 'visual.transformer.resblocks.0.attn.in_proj_bias', 'visual.transformer.resblocks.0.attn.out_proj.weight', 'visual.transformer.resblocks.0.attn.out_proj.bias', 'visual.transformer.resblocks.0.ln_1.weight', 'visual.transformer.resblocks.0.ln_1.bias', 'visual.transformer.resblocks.0.mlp.c_fc.weight', 'visual.transformer.resblocks.0.mlp.c_fc.bias', 'visual.transformer.resblocks.0.mlp.c_proj.weight', 'visual.transformer.resblocks.0.mlp.c_proj.bias', 'visual.transformer.resblocks.0.ln_2.weight', 'visual.transformer.resblocks.0.ln_2.bias', 'visual.transformer.resblocks.1.attn.in_proj_weight', 'visual.transformer.resblocks.1.attn.in_proj_bias', 'visual.transformer.resblocks.1.attn.out_proj.weight', 'visual.transf

In [5]:
categories = []
with open('/ckb-nfs/home/zcafego/imagenet_labels/synset_words.txt') as f:
    for line in f.readlines():
        line = line.strip()
        categories.append(' '.join(line.split(' ')[1:]).split(', ')[0])

In [6]:
def get_classifications_l14(img):
    preproc_img = l14_preprocessor(img).unsqueeze(0).to(device)
    tokens = clip.tokenize([f"a photo of a {category}" for category in categories]).to(device)

    with torch.no_grad():
        image_features = l14_model.encode_image(preproc_img)
        text_features = l14_model.encode_text(tokens)

        logits_per_image, _ = l14_model(image_features, text_features)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    
    print("Label probs: ", probs)

In [7]:
get_classifications_l14(img)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [1024, 3, 14, 14], but got 2-dimensional input of size [1, 768] instead

In [ ]:
# print(sorted(likenesses, key=lambda x: x[1], reverse=True))